# Первые пипы и скачивание датасета

In [1]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xjf LJSpeech-1.1.tar.bz2

--2021-12-19 15:55:28--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G  48.1MB/s    in 56s     

2021-12-19 15:56:24 (47.1 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [1]:
!pip install librosa

In [ ]:
!pip install torch==1.10.0+cu111 torchaudio==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install wandb

In [1]:
import os
import random
import torch.utils.data
import numpy as np
from librosa.util import normalize
from scipy.io.wavfile import read
from librosa.filters import mel as librosa_mel_fn

import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Tuple, Dict, Optional, List, Union
from itertools import islice
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from IPython import display
import itertools

import math

# Dataset

Датасет был честно скоммунизден из репозитория HIFI https://github.com/jik876/hifi-gan

In [2]:
from hifi.dataset import *

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
training_files = get_dataset_filelist('/content/LJSpeech-1.1', 
                                      '/content/drive/MyDrive/training.txt')
train_dataset  = MelDataset(training_files, 
                            segment_size=8192, 
                            n_fft =1024, 
                            num_mels = 80,
                            hop_size = 256, 
                            win_size = 1024, 
                            sampling_rate = 22050,  
                            fmin = 0, 
                            fmax = 8000 
                            )
dataloader = DataLoader(train_dataset,shuffle=True, batch_size=16)

# HIFI



## Генератор

In [5]:
from hifi.generator import *

## Дискриминатор

In [6]:
from hifi.discriminator import *

# Функции потерь

In [7]:
from hifi.losses import *

# Функции для тренировки 

## Логгирование аудио


In [8]:
from hifi.logging_functions import *

## Функция обучения без дискриминатора и c дискриминатором

In [9]:
from hifi.training_functions import *

# Обучение

In [10]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
h_u = 128
D_r = np.array([[[1, 1], [3, 1], [5, 1]], [[1, 1], [3, 1], [5, 1]], [[1, 1], [3, 1], [5, 1]]]) 
k_r = np.array([3, 7, 11])
k_u = np.array([16, 16, 4, 4])  
 
g = GeneratorModel(h_u=h_u, k_u=k_u,D_r=D_r,k_r=k_r)
     
g_opt = torch.optim.Adam(g.parameters(), lr=0.0002, betas=(0.8, 0.99))

In [11]:

mpd = MPDBlock()
msd = MSDBlock()


d_opt = torch.optim.Adam(itertools.chain(msd.parameters(), mpd.parameters()), lr=0.0002, betas=(0.8, 0.99))

In [13]:
import wandb
wandb.login()

wandb: Currently logged in as: kogana00 (use `wandb login --relogin` to force relogin)


True

In [14]:
wandb.init(project='hifi gan')

Это обучение итоговой модели:

In [ ]:
#Вывода нет, так как я перезапукала модель на разное к-во эпох. В итоге отучилось 35 эпох
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

n_epochs = 35
num_model = 81 #Это аргумент для сохранения модели, чтобы сохранялось в файл model_*num_model*.pt

train(g,
          mpd,
          msd,
          n_epochs,
          g_opt,
          d_opt,
          dataloader,
          num_model=num_model,
          logging=True
          )

# Тестирование 

In [72]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
h_u = 128
D_r = np.array([[[1, 1], [3, 1], [5, 1]], [[1, 1], [3, 1], [5, 1]], [[1, 1], [3, 1], [5, 1]]]) 
k_r = np.array([3, 7, 11])
k_u = np.array([16, 16, 4, 4])  
 
g = GeneratorModel(h_u=h_u, k_u=k_u,D_r=D_r,k_r=k_r)
                
loss_func =  nn.L1Loss()
opt = torch.optim.Adam(g.parameters(), lr=0.0002, betas=(0.8, 0.99))

In [73]:
g.load_state_dict(torch.load('/content/drive/MyDrive/hifi/g_81.pt'))#Last model

<All keys matched successfully>

In [ ]:
g.eval()
g = g.to(DEVICE)

In [70]:

n_fft =1024
num_mels = 80
hop_size = 256
win_size = 1024
sampling_rate = 22050
fmin = 0
fmax = 8000 


Тестирую

In [85]:
import torchaudio

In [88]:
test, sr = load_wav('/content/drive/MyDrive/test_wavs/audio_1.wav')
test = test / MAX_WAV_VALUE
test = normalize(test) * 0.95

test = torch.FloatTensor(test)
test = test.unsqueeze(0)
test.shape

test_mel = mel_spectrogram(test, 
                                    n_fft=n_fft,
                                    num_mels=num_mels,
                                    hop_size=hop_size,
                                    win_size=win_size,
                                    sampling_rate=sampling_rate,
                                    fmin = fmin,
                                    fmax=fmax).to(DEVICE)
g.eval()
g = g.to(DEVICE)
with torch.no_grad():
  test_wav = g(test_mel.to(DEVICE))
wav = test_wav.detach().squeeze(0).cpu()
torchaudio.save('hifi_audio_1.wav', wav, 22050)
wav = display.Audio(wav, rate=22050)
wav

In [89]:
test, sr = load_wav('/content/drive/MyDrive/test_wavs/audio_2.wav')
test = test / MAX_WAV_VALUE
test = normalize(test) * 0.95

test = torch.FloatTensor(test)
test = test.unsqueeze(0)
test.shape

test_mel = mel_spectrogram(test, 
                                    n_fft=n_fft,
                                    num_mels=num_mels,
                                    hop_size=hop_size,
                                    win_size=win_size,
                                    sampling_rate=sampling_rate,
                                    fmin = fmin,
                                    fmax=fmax).to(DEVICE)
g.eval()
g = g.to(DEVICE)
with torch.no_grad():
  test_wav = g(test_mel.to(DEVICE))
wav = test_wav.detach().squeeze(0).cpu()
torchaudio.save('hifi_audio_2.wav', wav, 22050)
wav = display.Audio(wav, rate=22050)
wav

In [90]:
test, sr = load_wav('/content/drive/MyDrive/test_wavs/audio_3.wav')
test = test / MAX_WAV_VALUE
test = normalize(test) * 0.95

test = torch.FloatTensor(test)
test = test.unsqueeze(0)
test.shape

test_mel = mel_spectrogram(test, 
                                    n_fft=n_fft,
                                    num_mels=num_mels,
                                    hop_size=hop_size,
                                    win_size=win_size,
                                    sampling_rate=sampling_rate,
                                    fmin = fmin,
                                    fmax=fmax).to(DEVICE)
g.eval()
g = g.to(DEVICE)
with torch.no_grad():
  test_wav = g(test_mel.to(DEVICE))
wav = test_wav.detach().squeeze(0).cpu()
torchaudio.save('hifi_audio_3.wav', wav, 22050)
wav = display.Audio(wav, rate=22050)
wav